In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import pdb

In [4]:
#import zillow_median$/ft

df_rent = pd.read_csv('../data/zillow/city/City_MedianRentalPricePerSqft_AllHomes.csv')



In [5]:
df_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1606 entries, 0 to 1605
Columns: 111 entries, RegionName to 2018-10
dtypes: float64(106), int64(1), object(4)
memory usage: 1.4+ MB


In [6]:
df_own = pd.read_csv('../data/zillow/city/City_MedianValuePerSqft_AllHomes.csv', header=0)

In [7]:
df_rent.columns.values

array(['RegionName', 'State', 'Metro', 'CountyName', 'SizeRank',
       '2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
       '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05

In [8]:
#Load Consumer Expenditure

base_ce_path = '../data/bls_ce/msa/'

df_west = pd.read_excel('../data/bls_ce/msa/west.xlsx', header=2)

In [9]:


class COLDataLoader():
    #A Class for loading data for the Cost of living calculator.
    #Instantiate with the path to the Zillow city csvs and Consumer Expenditure xlsx
    
    def __init__(self, zillow_path, ce_path, state_tax_path):
        self.zillow_path = zillow_path
        self.ce_path = ce_path
        self.state_tax_path = state_tax_path
        self.df_zil_own = None
        self.df_zil_rent = None
        self.df_ce = None
        self.df_state_tax = None
        
    def load(self):
        #Load Zillow
        self.df_zil_own = pd.read_csv(os.path.join(self.zillow_path, 'City_MedianValuePerSqft_AllHomes.csv'), header=0)
        self.df_zil_rent = pd.read_csv(os.path.join(self.zillow_path,'City_MedianRentalPricePerSqft_AllHomes.csv'))
        
        #Load CE
        file_list = self.get_file_list(self.ce_path)
        list_df = self.get_dataframes(file_list)
        self.df_ce = self.merge_dataframes(list_df)
        
        #Load State Tax
        self.get_state_tax()
        
    def remove_non_numeric(self,x):
        try:
            return float(x)
        except:
            return float(''.join([d for d in str(x) if d.isnumeric()]))
        
    def get_state_tax(self):
        path = os.path.join(self.state_tax_path, os.listdir(self.state_tax_path)[0])
        self.df_state_tax = pd.read_csv(path, na_values='None')
        #Make infinite range
        self.df_state_tax.loc[self.df_state_tax.incomeNotGreaterThan.isna(),'incomeNotGreaterThan'] = np.inf
        #Fill other rates with 0
        self.df_state_tax.iloc[:,-3:-1]= self.df_state_tax.iloc[:,-3:-1].fillna(0)
        #Remove Non-numeric characters from numeric columns
        self.df_state_tax.iloc[:,-3:] = self.df_state_tax.iloc[:,-3:].applymap(self.remove_non_numeric)
        
    
    def get_file_list(self, base_path):
        #Input: String - folder path to xlsx files
        #Output: List of Strings - paths to the files
        files = os.listdir(base_path)
        file_list = []
        for file in files:
            if not file.startswith('~'):
                file_path = os.path.join(base_path, file)
                file_list.append(file_path)
        return file_list

    def get_dataframes(self, file_list):
        #Input: List of Strings - paths to xlsx files
        #Output: List of pd dataframes
        list_df = []
        for file in file_list:
            df = pd.read_excel(file, header=2)
            df = df.loc[~df.iloc[
                
                
                :,0].isna(),:]
            df.set_index('Item', inplace=True)
            list_df.append(df)
        return list_df

    def merge_dataframes(self, list_df):
        #Input: list of pd dataframes:
        
        #Output: dataframe joined columnwise by index.
        return pd.concat(list_df, axis=1, ignore_index=False)

In [10]:
base_ce_path = '../data/bls_ce/msa/'
base_zil_path = '../data/zillow/city/'
base_tax_path = '../data/state_tax/'
data = COLDataLoader(base_zil_path, base_ce_path, base_tax_path)
data.load()

/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py:76: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [16]:
os.path.join(base_ce_path, '../')

'../data/bls_ce/msa/../'

In [11]:
class COLCalculator():
    #Cost of Living Calculator
    #Instantiate with the path to the Zillow city csvs and Consumer Expenditure xlsxs
    def __init__(self, zillow_path, ce_path, state_tax_path):
        self.data = COLDataLoader(zillow_path, ce_path, state_tax_path)
        self.data.load()
        
    def calculate(self, gross_income, city, state, married=False):
        #Calculation function: returns a dict (or json) with the following values
        #income_taxes (Tuple): (Float: Approx amount of taxes paid, Float: Approx. Effective Tax Rate)
        #housing_rent (Tuple): (Int: sqft affordable at the housing % ratio, Float: Ratio of housing/income)
        #housing_own (Tuple): (Int: sqft affordable at the housing % ratio, Float: Ratio of housing/income)
        tax_amount, tax_rate, net_income = self._get_taxes(gross_income, city, state, married)
#         housing_rent, housing_own = self._get_housing(gross_income, city, state)
        return {'taxes': tuple((tax_amount, tax_rate, net_income))}
    
    # Tax Calculation  - Federal and State
    def _get_taxes(self, gross_income, city, state, married):
        #returns tax amount, tax %, and net income
        fed_tax, fed_rate = self._calc_fed_tax(gross_income, married)
        state_tax, state_rate = self._calc_state_tax(gross_income, state, married)
        pdb.set_trace()
        total_tax = fed_tax + state_tax
        net_income = gross_income - total_tax
        effective_tax = total_tax/gross_income

        return total_tax, effective_tax, net_income


    
    def _calc_fed_tax(self, gross_income, married):
        #tax withholding from https://www.irs.gov/pub/irs-pdf/n1036.pdf
        #Returns Tax Amount and rate used to calculate withholding
        single_rate = np.array([[3800, 0, .1],
                                [13500, 970,  .12],
                                [43275, 4543,  .22],
                                [88000, 14382, .24],
                                [164525, 32748,  .32],
                                [207900, 46628, .35],
                                [514100, 153799, .37]]
                                )

        single_rate = np.hstack((single_rate, 
                                np.append(single_rate[1:,0],np.inf).reshape(-1,1)
                                ))

        married_rate = np.array([[11800, 0, .1],
                                 [31200, 1940, .12],
                                 [90750, 9086, .22],
                                 [180200, 28765, .24],
                                 [333250, 65497, .32],
                                 [420000, 93257, .35],
                                 [624150, 164710, .37]
                                ])

        married_rate = np.hstack((married_rate, 
                                np.append(married_rate[1:,0],np.inf).reshape(-1,1)
                                ))
        rate = single_rate
        if married:
            rate = married_rate

        mask = np.logical_and(rate[:,0]<=gross_income, rate[:,-1]>gross_income)

        mask = np.logical_and(rate[:,0]<=gross_income, rate[:,-1]>gross_income)
        if mask.any():
            threshold, base_tax, rate = rate[mask][0][0], rate[mask][0][1], rate[mask][0][2]
            pdb.set_trace()
            return ((base_tax + (gross_income-threshold)*rate), rate)
        else:
            return (0, 0)

        
    def _calc_state_tax(self,gross_income, state, married):
        #Returns State Tax Amount and Rate used to calculate the amount
        income = gross_income - 12000
        if married:
            income = gross_income - 24000
        tax_rates = self.data.df_state_tax.loc[self.data.df_state_tax.stateAbbr == state]
        mask = np.logical_and(tax_rates.iloc[:,-2].values<gross_income, tax_rates.iloc[:,-1].values>=gross_income)
        rate = tax_rates.loc[mask,'incomeTaxRate'].values[0]
        return (income*rate, rate)
    
    def _get_housing(self, gross_income, city, state):
        #Returns two tuples
        #housing_rent (Tuple): (Int: sqft affordable at the housing % ratio, Float: Ratio of housing/income)
        #housing_own (Tuple): (Int: sqft affordable at the housing % ratio, Float: Ratio of housing/income)
        pass
    
    def _get_transporation(self, gross_income, city):
        #Returns amount of transportation ?
        pass
    
    def _get_lifestyle(self, gross_income, city):
        pass
    
    def _get_savings(self, gross_income, city):
        pass

In [12]:
calculator = COLCalculator(base_zil_path, base_ce_path, base_tax_path)


/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py:76: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [13]:
print(calculator.calculate(150000, 'San Francisco', 'CA'))

> <ipython-input-11-8561a6486968>(69)_calc_fed_tax()
-> return ((base_tax + (gross_income-threshold)*rate), rate)
(Pdb) continue
> <ipython-input-11-8561a6486968>(23)_get_taxes()
-> total_tax = fed_tax + state_tax
(Pdb) continue
{'taxes': (42096.0, 0.28064, 107904.0)}


In [74]:
def calc_state_tax(gross_income, state):
    tax_rates = data.df_state_tax.loc[data.df_state_tax.stateAbbr == state]
    mask = np.logical_and(tax_rates.iloc[:,-2].values<gross_income, tax_rates.iloc[:,-1].values>=gross_income)
    
    return tax_rates.loc[mask,'incomeTaxRate'].values

In [101]:
calc_state_tax(100000, 'CA')

array([0.093])

In [73]:
data.df_state_tax.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 6 columns):
id                      211 non-null int64
stateAbbr               211 non-null object
stateName               211 non-null object
incomeTaxRate           211 non-null float64
incomeGreaterThan       211 non-null float64
incomeNotGreaterThan    211 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 10.0+ KB
